### Test data set

> Length = 1500bps   
> Flank  = 100bps   
> Minin  = 50bps   
> Minex  = 50bps   
> Maxs   = 3

For later double check about the result   
The sequence we used:    

seq = '''AGGTTTTCAGTCTGCGTATCTAAATCAGCATTAATAACAATGAGGCGAGGATAACAGCAA
TCGGGCCTCTAGTCCTTTTTGCTGGCCAAAGTGAACGTCAGCTAGGTAACGTTGCGCATT
CAAAGACGAGTAGTGTCGCGAAACAAATAGTTATCAGTGCTACAATTACATGCGCAACAA
TTGAGGAGTTAAGGCACTACCACGGAACAGAGACCGTGTGCCAGGAGAAGTGGTACGATT
GCTCCTGACAGTTGCCGACTCCGGGCGCATTGACTGTCACTTCGAAGTCATGGCACCGAG
GGGCTAGCCGGACCTCGGTAGCAGGCCCGTAGTCCTCCAGAGAGACACAGGTTTCGTACT
CCTAGTGAGAAATAATATCACCTGATCCCATGTTCACTAGGACCGTCACTAGCTTGCCTA
TGTGGCACAACGCAGGGAGCCAGATACAAATGAATCGCTAGCGGTTGCTGGCCCATGCCT
GGGCTATCTTAGTGACATTGGGCTGGAAAGGACAGTGCTGAGTATGGATGCCAGAAAAAA
GCCTAGACCGTGTGGGTGCTCGGACCGATCACCTCATTCACACACGACTTCATGCCGACT
TCCTCACGCTTGGTGGTATTCTCAGCGATATGGGTAGAATTCCCTGCTCAAGCCGGTGGT
TTATTGTGTTTGCGCTTCGGCAGGACGCTGCCTCGTGGGATCTCGATTTTAGCTGTGCCA
ATCAGCAATTTAGGAGAGCATATACATTGTTAACAACCCTATTTTACCTTACTCAGTAGG
TGCTACACTCTTTTACGTTAATGCCGTTATCCTCATAGACAGGTTACGTGGCGTGCGGCT
AGCGAGAGTGAGAAGTTGTCTTCACAGGATGTTTAGCTAAAGACTCACCGCAGCTCGTTC
TCAACGACGCTTGTTCACCGAAAAAAGTAAAGTGAGTTCCTACCCTTTGATAATATGTAT
TGGCGTGCTTGCGACGCCTGGCCTCAACAGATGCCCAGCTACCCTCATGGGGGGACTCGA
CACTCTACCCCTATTGGGACAGGTGAGACCTTTGGTCCTAGATGTTCTTCTCTCTCTAAG
CTCAGCTTATAATGCCGGTTAATTTGGCAACCCCCGTAAGTTGCCAGTTACTGATGTACG
GAAGGAGCCCTCCGTTGGGCCCGAAAACTCGATCGGTATGTCGGCCGATCCCCCCCCCAA
ACGAAAACAAGTTTCATTTAGTAGAGACGTACACTAACTGAAGTTGGGTCGACACGGCAT
TGCCGACAGTATTCTTTGTGTTCATCCTTCACGCAATAGCTCTAGGTCACCTATTGGGAG
TACGCAGTGACGATAAGTAGTGCACTAATTAGTACAGTCAGAGTCGTCATCGGAGCACTG
CAGCCTTGACGCGCTCATGCTTCTCTAGTCCTTGGACTTGGAGCCCACGTGTAGTGCTAG
GCACCCAAACAGTGATGTAGCCGAAACAATATATTTCCTGGATGACTCGTCCTTGAATCC'''

### Idea1
using different nested loop so we could do more precise unsatisfied intron and exon remove

In [ ]:
def gtag_sites(seq, flank, minex):
    dons = []
    accs = []
    for i in range(flank + minex, len(seq) -flank -minex -1):
        if seq[i:i+2]   == 'GT': dons.append(i)
        if seq[i-1:i+1] == 'AG': accs.append(i)
    return dons, accs

using original `gtag extracting` doesn't work that well
original one gives out dons and accs in version of list

> dons = []   
> accs = []   

since for using up a for loop to make all possible combination, we also need the enumerate so that each time when the nested loop pass down to the next combination
there would require more memory and time complexity for computer to keep track of those numbers

In [ ]:
dons, accs = gtag_sites(seq, 100, 50)

for i1, d1 in enumerate(dons):
    if d1 - 100 -1 < 50: continue
    
    for j1, a1 in enumerate(accs):
        if a1 - d1 < 50: continue
        
        for i2, d2 in enumerate(dons[i1:]):
            if d2 - a1 < 50: continue
            
            for j2, a2 in enumerate(accs[j1:]):
                if a2 - d2 < 50: continue
                
                for i3, d3 in enumerate(dons[i2:]):
                    if d3 - a2 < 50: continue
            
                    for a3 in accs[j2:]:
                        if a3 - d3 < 50: continue
                        if a3 - 100 - 1 < 50:  continue
                        print(d1, a1, d2, a2, d3, a3)

Performance: 110.64s user 0.56s system 99% cpu 1:51.22 total

### Idea2   
using dictionary instead of list for donor and acceptor site

In [ ]:
def gtag_sites(seq, flank, minex):
    dons = {}
    accs = {}
    d = 0
    a = 0
    for i in range(flank + minex, len(seq) -flank -minex -1):
        if seq[i:i+2]   == 'GT': 
            d += 1
            dons[d] = i
        if seq[i-1:i+1] == 'AG': 
            a += 1
            accs[a] = i
    return d, a, dons, accs

in the revised version, the gtag sites use a dictionary for extracting informaton   

> dons = { `index`(int) : `donor site`(int)...}   
> accs = { `index`(int) : `accep site`(int)...}   

it would generate better performance when extracting ideal output and faster through looping

In [ ]:
flank = 100
minex = 50
minin = 50
d, a, dons, accs = gtag_sites(seq, flank, minex)

for d1 in range(1, d+1):
    if dons[d1] < minex + flank + 1: continue
    
    for a1 in range(1, a+1):
        if accs[a1] < dons[d1] + minin: continue

        for d2 in range(d1+1, d+1):
            if dons[d2] < accs[a1] + minex: continue

            for a2 in range(a1+1, a+1):
                if accs[a2] < dons[d2] + minin: continue

                for d3 in range(d2+1, d+1):
                    if dons[d3] < accs[a2] + minex: continue
                    
                    for a3 in range(a2+1, a+1):
                        if accs[a3] < minin + dons[d3]: continue
                        if accs[a3] + flank + minex > len(seq):  continue
                        print( dons[d1], accs[a1], dons[d2], accs[a2], dons[d3], accs[a3] )

Performance: 112.95s user 0.50s system 99% cpu 1:53.47 total

here is an simple test to proof baisc logic of the nest loop , they would generate all possible combination based on these idea

In [ ]:
for i in range(1,4+1):
    for j in range(i+1,5):
        print(i, j)

reflect on this version:
+ Pros   
  - Easier removing untargetted output   
    - Since everything is more specified, it is easier for removing small introns and exons   
    - we don't external def function there
  - Less time complexity
    - approximately `O(nlogn)` would be larger if max_exon comes more than 3   
    - since the combination n is collpasing through out the loop   
+ Cons
  - nobody wants such big truck of garbage code
    - max_exon could not even fix there
    - this could actually be fixed by using recursion method

## Idea 3
space complexity for time complexity

all previous idea make action on improving the overall `time complexity` for optimization   
the next trail for this idea is consuming more `space complexity` for better faster output    
below is the single proof for our idea

In [8]:
for i, j in zip( range(1,6), range(5,0,-1)):
    print(i, j)

1 5
2 4
3 3
4 2
5 1


### Coding

btw the key question is; **which one are we gonna making it reversely, coupled with each for loop?**   
we have two choices:
- [solution1]
  - make similar stuff runnning spontaneouly btw from the end 
  - zip one donor site from beginning of donor site and donor site from end of donor site
- ~~[solution2]~~
  - make different stuff running the acceptor site from the end
  - zip another acceptor site from end of acceptor site 
  - do the question at the same time btw completely in reverse order

### Why solution 2 not working?

- Solution 2 could not capable of not generating same result(wasting time)
- Solution 2 is messy whenever we want to code it
  - here is how it looks like when max combination aka n=1, this is horrible


In [ ]:
fgc = True
rgc = True
fgv = None
rgv = None

for fd1 in range(1, d+1):
    # update gate control
    fgc = dons[fd1] >= minex + flank + 1
    ra1 = len(accs) - fd1 + 1
    rgc = accs[ra1] <= len(seq) - flank - minex
    if not(fgc and rgc): continue
    # track global value
    fgv, rgv = dons[fd1], accs[ra1]

    for fa1 in range(1, a+1):
        # update gate control
        if fgc: fgc = accs[fa1] >= fgv + minin
        rd1 = len(dons) - fa1 + 1
        if rgc: rgc = dons[rd1] <= rgv - minin
        if not(fgc and rgc): continue
        # track global value
        fgv, rgv = accs[fa1], dons[rd1]

### Solution1
We do permutation from beg to end while coupling with permutation from end to beg

In [ ]:
def gtag_sites(seq, flank, minex):
    dons = {}
    accs = {}
    d = 0
    a = 0
    for i in range(flank + minex, len(seq) -flank -minex -1):
        if seq[i:i+2]   == 'GT': 
            d += 1
            dons[d] = i
        if seq[i-1:i+1] == 'AG': 
            a += 1
            accs[a] = i
    return d, a, dons, accs

seq = '''AGGTTTTCAGTCTGCGTATCTAAATCAGCATTAATAACAATGAGGCGAGGATAACAGCAA
TCGGGCCTCTAGTCCTTTTTGCTGGCCAAAGTGAACGTCAGCTAGGTAACGTTGCGCATT
CAAAGACGAGTAGTGTCGCGAAACAAATAGTTATCAGTGCTACAATTACATGCGCAACAA
TTGAGGAGTTAAGGCACTACCACGGAACAGAGACCGTGTGCCAGGAGAAGTGGTACGATT
GCTCCTGACAGTTGCCGACTCCGGGCGCATTGACTGTCACTTCGAAGTCATGGCACCGAG
GGGCTAGCCGGACCTCGGTAGCAGGCCCGTAGTCCTCCAGAGAGACACAGGTTTCGTACT
CCTAGTGAGAAATAATATCACCTGATCCCATGTTCACTAGGACCGTCACTAGCTTGCCTA
TGTGGCACAACGCAGGGAGCCAGATACAAATGAATCGCTAGCGGTTGCTGGCCCATGCCT
GGGCTATCTTAGTGACATTGGGCTGGAAAGGACAGTGCTGAGTATGGATGCCAGAAAAAA
GCCTAGACCGTGTGGGTGCTCGGACCGATCACCTCATTCACACACGACTTCATGCCGACT
TCCTCACGCTTGGTGGTATTCTCAGCGATATGGGTAGAATTCCCTGCTCAAGCCGGTGGT
TTATTGTGTTTGCGCTTCGGCAGGACGCTGCCTCGTGGGATCTCGATTTTAGCTGTGCCA
ATCAGCAATTTAGGAGAGCATATACATTGTTAACAACCCTATTTTACCTTACTCAGTAGG
TGCTACACTCTTTTACGTTAATGCCGTTATCCTCATAGACAGGTTACGTGGCGTGCGGCT
AGCGAGAGTGAGAAGTTGTCTTCACAGGATGTTTAGCTAAAGACTCACCGCAGCTCGTTC
TCAACGACGCTTGTTCACCGAAAAAAGTAAAGTGAGTTCCTACCCTTTGATAATATGTAT
TGGCGTGCTTGCGACGCCTGGCCTCAACAGATGCCCAGCTACCCTCATGGGGGGACTCGA
CACTCTACCCCTATTGGGACAGGTGAGACCTTTGGTCCTAGATGTTCTTCTCTCTCTAAG
CTCAGCTTATAATGCCGGTTAATTTGGCAACCCCCGTAAGTTGCCAGTTACTGATGTACG
GAAGGAGCCCTCCGTTGGGCCCGAAAACTCGATCGGTATGTCGGCCGATCCCCCCCCCAA
ACGAAAACAAGTTTCATTTAGTAGAGACGTACACTAACTGAAGTTGGGTCGACACGGCAT
TGCCGACAGTATTCTTTGTGTTCATCCTTCACGCAATAGCTCTAGGTCACCTATTGGGAG
TACGCAGTGACGATAAGTAGTGCACTAATTAGTACAGTCAGAGTCGTCATCGGAGCACTG
CAGCCTTGACGCGCTCATGCTTCTCTAGTCCTTGGACTTGGAGCCCACGTGTAGTGCTAG
GCACCCAAACAGTGATGTAGCCGAAACAATATATTTCCTGGATGACTCGTCCTTGAATCC'''

flank = 100
minex = 50
minin = 50
d, a, dons, accs = gtag_sites(seq, flank, minex)

# fdn = forward donor site n    fan = forward acceptor site n
# ran = reverse acceptor site n rdn = reverse donor site n
# fgc = forward gate control     rgc = reverse gate control
# fgv = forward global value    rgv = reverse global value

fgc = True
rgc = True
fgv = None
rgv = None

for fd1 in range(1, d+1):
    # exit output
    rd1 = len(dons) - fd1 + 1
    if rd1 < fd1: break
    # collecting boolean
    fgc = dons[fd1] >= minex + flank + 1
    rgc = dons[rd1] >= minex + flank + 1
    if not(fgc or rgc): continue
    
    for fa1 in range(1, a+1):
        # skip current loop
        if   fgc and rgc:
            fgc1 = accs[fa1] >= dons[fd1] + minin
            ra1 = len(accs) - fa1 + 1
            rgc1 = accs[ra1] >= dons[rd1] + minin
            if not(fgc1 or rgc1): continue 
        elif fgc and not rgc:
            fgc1 = accs[fa1] >= dons[fd1] + minin
            if not fgc1: continue
            rgc1 = False
        elif rgc and not fgc:
            ra1 = len(accs) - fa1 + 1
            rgc1 = accs[ra1] >= dons[rd1] + minin
            if not rgc1: continue
            fgc1 = False
        
        for fd2 in range(fd1 + 1, d+1):
            if fgc1 and rgc1: 
                fgc2 = dons[fd2] >= accs[fa1] + minin
                rd2 = len(dons) - fd2 + 1
                rgc2 = dons[rd2] >= accs[ra1] + minin
                if not (fgc2 or rgc2): continue 
            elif fgc1 and not rgc1:
                fgc2 = dons[fd2] >= accs[fa1] + minin
                if not fgc2: continue
                rgc2 = False    
            elif rgc1 and not fgc1:
                rd2 = len(dons) - fd2 + 1
                rgc2 = dons[rd2] >= accs[ra1] + minin
                if not rgc2: continue
                fgc2 = False
    
            for fa2 in range(fa1 + 1, a+1):
                if fgc2 and rgc2:
                    fgc3 = accs[fa2] >= dons[fd2] + minin
                    ra2  = len(accs) - fa2 +1
                    rgc3 = accs[ra2] >= dons[rd2] + minin
                    if not (fgc3 or rgc3): continue
                elif fgc2 and not rgc2:
                    fgc3 = accs[fa2] >= dons[fd2] + minin
                    if not fgc3: continue
                    rgc3 = False
                elif rgc2 and not fgc2:
                    ra2  = len(accs) - fa2 +1
                    rgc3 = accs[ra2] >= dons[rd2] + minin
                    if not rgc3: continue
                    fgc3 = False
    
                for fd3 in range(fd2 + 1, d+1):
                    if   fgc3 and rgc3:
                        fgc4 = dons[fd3] >= accs[fa2] + minin
                        rd3 = len(dons) - fd3 + 1
                        rgc4 = dons[rd3] >= accs[ra2] + minin
                        if not (fgc4 or rgc4): continue
                    elif fgc3 and not rgc3:
                        fgc4 = dons[fd3] >= accs[fa2] + minin
                        if not fgc4: continue
                        rgc4 = False
                    elif rgc3 and not fgc3:
                        rd3 = len(dons) - fd3 + 1
                        rgc4 = dons[rd3] >= accs[ra2] + minin
                        if not rgc4: continue
                        fgc4 = False
                    
                    for fa3 in range(fa2 + 1, a+1):
                        if fgc4 and rgc4:
                            fgc5 = accs[fa3] >= minin + dons[fd3]
                            fgc6 = accs[fa3] + flank + minex <= len(seq)
                            ra3 = len(accs) - fa3 + 1
                            rgc5 = accs[ra3] >= minin + dons[rd3]
                            rgc6 = accs[ra3] + flank + minex <= len(seq)
                            if fgc5 and fgc6: print( dons[fd1], accs[fa1], dons[fd2], accs[fa2], dons[fd3], accs[fa3] )
                            if rgc5 and rgc6: print( dons[rd1], accs[ra1], dons[rd2], accs[ra2], dons[rd3], accs[ra3] )
                            if not(fgc5 and fgc6) and not(rgc5 and rgc6): continue
                        elif fgc4 and not rgc4:
                            fgc5 = accs[fa3] >= minin + dons[fd3]
                            fgc6 = accs[fa3] + flank + minex <= len(seq)
                            if fgc5 and fgc6: print( dons[fd1], accs[fa1], dons[fd2], accs[fa2], dons[fd3], accs[fa3] )
                            else: continue
                        elif rgc4 and not fgc4:
                            ra3 = len(accs) - fa3 + 1
                            rgc5 = accs[ra3] >= minin + dons[rd3]
                            rgc6 = accs[ra3] + flank + minex <= len(seq)
                            if rgc5 and rgc6: print( dons[rd1], accs[ra1], dons[rd2], accs[ra2], dons[rd3], accs[ra3] )
                            else: continue

## Check

with 1524 basepair input with 62 donor site and 66 acceptor site   
flank length = 150   
min ex = 100   
min in = 100   

> `1515018 combination` from idea2   
> `1515018 combination` from solution1 

time consumed for each on   

> idea2: 3.16s user 0.04s system 99% cpu 3.227 total
> solution1: 5.12s user 0.06s system 99% cpu 5.195 total

This proof that solution 2 would generate the correct result

with 1524 basepair input with 62 donor site and 66 acceptor site    
flank length = 100    
min ex = 50   
min in = 50   
There is unpair with idea2 and solution1 values; btw solution 1 are more diversed, i guess there is sort of limitation with current computer
> `Current hypothesis`: solution 1 works better with larger input and idea2 works wiht smaller input.

## Backtrackcing Algorithm

it is something similar to idea of iteration but way harder   
this would help in overall code neatness for this algorithm
this gonna work on the idea 1 ; input is two list and finish this with a sort of tree 

> Estimated time complexity O(2*2^n)   
> Estimated Space complexity O(n)   

below is simple example for how recursion could do on calculating the factorial of something

In [16]:
nums = [1,2,3]

def program(nums: list[int] ):
    n = len(nums)
    res, sol = [], []

    def backtrack(i):
        if i == n:
            res.append(sol[:])
            return
        
        backtrack(i+1)

        sol.append(nums[i])
        backtrack(i+1)
        sol.pop()

    backtrack(0)
    return res

print( program(nums) )

[[], [3], [2], [2, 3], [1], [1, 3], [1, 2], [1, 2, 3]]


The performance of this for finding combination up to eight is:
> 0.02s user 0.01s system 90% cpu 0.034 total.   

Since the overall time complexity are quite simple.  
Approximately reach to `O(2^n)`   
number of going through + number of going back

Let's make this program into the version that we are gonna used.   
this time, we wanna make the overall program able to run in a tree with `len(list) input != depth` we want    
> basically it's the fist step for everything    
> since in the real part, we need to do two list with depth of 3 or 4   

In [ ]:
nums = [1,2,3,4,5]

def program(nums: list[int], n: int ):
    res, sol = [], []

    def backtrack(i):
        if i == n:
            res.append(sol[:])
            return
        
        if len(nums) - i < n - len(sol): return

        for j in range(i, len(nums)):
            sol.append(nums[j])
            backtrack(i+1)
            sol.pop()

    backtrack(0)
    return res

print( program(nums, 3) )

In this version, we have two different trait:

+ A for loop for putting all possible nodes instead of binary tree
+ An exit boolean condition if there is no nodes available for reaching the designated depth.

In the next on, we need to exclude every possible combination that don't follow the rule that we wish every number poped in is larger than what's before.

In [21]:
nums = [1,2,3,4,5]

def program(nums: list[int], n: int ):
    res, sol = [], []

    def backtrack(i):
        if i == n:
            res.append(sol[:])
            return
        
        if len(nums) - i < n - len(sol): return

        for j in range(i, len(nums)):
            if sol:
                if nums[j] <= sol[-1]: continue
            sol.append(nums[j])
            backtrack(i+1)
            sol.pop()

    backtrack(0)
    return res

print( program(nums, 3) )

[[1, 2, 3], [1, 2, 4], [1, 2, 5], [1, 3, 4], [1, 3, 5], [1, 4, 5], [2, 3, 4], [2, 3, 5], [2, 4, 5], [3, 4, 5]]


Now, technically, this one is a sort of **min heap**.   
What we did there is one stuff.   

> We make a nested condition to check whether there is number in the solution space is there always the number that are larger than the previous node?    

Basically, this line delete those nodes that are unsatifised our condition for a valid output. 
Moreover,
>In the actual programmed output, we also need to count small-intron and small-exon whoever are stuff that are not 
>satisfied the basic condition of introns and exons.    
>We shall count them globally before cutting off those nodes.


required for next version     
- select list1 as node whenever the depth is odd   
- select list2 as node whenever the depth is even    
- while also maintaining the condition that this is a min heap   

In [ ]:
nums1 = [1,3,5,7,9]
nums2 = [2,4,6,8,10]

def program(nums1: list[int], nums2: list[int] , n: int ):
    res, sol = [], []

    def backtrack(i):
        if i == n:
            res.append(sol[:])
            return
        
        if (n - i) % 2 == 0:
            for n2 in range(i, len(nums2) ):
                if sol:
                    if nums2[n2] <= sol[-1]: continue
                sol.append(nums2[n2])
                backtrack(i+1)
                sol.pop()
        else: 
            for n1 in range(i, len(nums1) ):
                if sol:
                    if nums1[n1] <= sol[-1]: continue
                sol.append(nums1[n1])
                backtrack(i+1)
                sol.pop()

    backtrack(0)
    return res

print( program(nums1, nums2, 4) )

In this version, we successfully turn the original backtrack algorithm into what we want.  

> nums2 == `donor    site`    
> nums1 == `acceptor site`

The next version gonna be the one that matches our needs.

In [ ]:
def gtag_sites(seq, flank, minex):
    dons = []
    accs = []
    for i in range(flank + minex, len(seq) -flank -minex -1):
        if seq[i:i+2]   == 'GT': dons.append(i)
        if seq[i-1:i+1] == 'AG': accs.append(i)
    return dons, accs

flank = 100
minex = 25
minin = 25
dons, accs = gtag_sites(seq, flank, minex)

def program(accs: list[int], dons: list[int] , n: int ):
    res, sol = [], []
    info = {
		'trials' : 0,
		'donors': len(dons),
		'acceptors': len(accs),
		'short_intron': 0,
		'short_exon': 0,
	}
    def backtrack(i):
        if i == n:
            if   sol[-1] + minex + flank > len(seq): return
            elif sol[-1] + flank < len(seq) + minex: return
            res.append(sol[:])
            return
        
        if (n - i) % 2 == 0:
            for n2 in range(i, len(dons) ):
                info['trials'] += 1
                if sol:
                    if dons[n2] <= sol[-1] + minex:
                        info['short_exon'] += 1
                        continue
                sol.append(dons[n2])
                backtrack(i+1)
                sol.pop()
        else: 
            for n1 in range(i, len(accs) ):
                info['trials'] += 1
                if sol:
                    if accs[n1] <= sol[-1] + minin:
                        info['short_intron'] += 1
                        continue
                sol.append(accs[n1])
                backtrack(i+1)
                sol.pop()

    backtrack(0)
    return res, info

## Dynamic Programming
it inherit idea of `recursion`, `divide and conquer`   
using smaller part of question to solve question itself    
since there is a patter that we use for this question:     
here is the pattern